In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.api import VAR
import pmdarima as pm

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("../data/historic_demand_2009_2023.csv")

In [4]:
elec_df = df[192_816: 194_788]

In [5]:
elec_df = elec_df[["settlement_date", "embedded_wind_generation"]]

In [6]:
elec_df.head()

,settlement_date,embedded_wind_generation
192816,2020-01-01,1244
192817,2020-01-01,1188
192818,2020-01-01,1156
192819,2020-01-01,1125
192820,2020-01-01,1106


In [7]:
wind_df = pd.read_csv("../data/Hull_2020-01-01_to_2020-02-10.csv")
wind_df["windgust"].tail()


979    53.6
980    49.7
981    57.8
982    65.0
983    78.4
Name: windgust, dtype: float64

In [8]:
wind_df = wind_df[["datetime","windgust", "windspeed"]]

In [9]:
wind_df['windgust'] = wind_df['windgust'].fillna(0)

In [10]:
wind_df.head()

,datetime,windgust,windspeed
0,2020-01-01T00:00:00,0.0,1.7
1,2020-01-01T01:00:00,0.0,4.1
2,2020-01-01T02:00:00,0.0,4.0
3,2020-01-01T03:00:00,0.0,4.3
4,2020-01-01T04:00:00,0.0,4.3


In [11]:
wind_df_long = pd.DataFrame(np.repeat(wind_df.values, 2, axis=0))
wind_df_long.columns = wind_df.columns
wind_df_long.head()

,datetime,windgust,windspeed
0,2020-01-01T00:00:00,0.0,1.7
1,2020-01-01T00:00:00,0.0,1.7
2,2020-01-01T01:00:00,0.0,4.1
3,2020-01-01T01:00:00,0.0,4.1
4,2020-01-01T02:00:00,0.0,4.0


In [12]:
wind_df_long["windspeed"] = wind_df_long["windspeed"]*120

In [13]:
wind_df_long.head()

,datetime,windgust,windspeed
0,2020-01-01T00:00:00,0.0,204.0
1,2020-01-01T00:00:00,0.0,204.0
2,2020-01-01T01:00:00,0.0,492.0
3,2020-01-01T01:00:00,0.0,492.0
4,2020-01-01T02:00:00,0.0,480.0


In [14]:
test_begin_point = 1475

In [15]:
endog_train = elec_df["embedded_wind_generation"][:test_begin_point].to_numpy()
endog_train = np.array(endog_train, dtype=float)
len(endog_train)

1475

In [16]:
endog_test_df = elec_df["embedded_wind_generation"][test_begin_point:-4]
endog_test = np.array(endog_test_df, dtype=float)
len(endog_test)

493

In [17]:
exog_train = wind_df_long["windspeed"][:test_begin_point].to_numpy()
exog_train = np.array(exog_train, dtype=float)
exog_train = exog_train
len(exog_train)

1475

In [18]:
exog_test = wind_df_long["windspeed"][test_begin_point:].to_numpy()
exog_test = np.array(exog_test, dtype=float)
exog_test = exog_test
len(exog_test)

493

In [19]:
wind_df_long["datetime"]

0       2020-01-01T00:00:00
1       2020-01-01T00:00:00
2       2020-01-01T01:00:00
3       2020-01-01T01:00:00
4       2020-01-01T02:00:00
               ...         
1963    2020-02-10T21:00:00
1964    2020-02-10T22:00:00
1965    2020-02-10T22:00:00
1966    2020-02-10T23:00:00
1967    2020-02-10T23:00:00
Name: datetime, Length: 1968, dtype: object

In [20]:
endog_train = pd.DataFrame(endog_train.astype(float),columns=["endog"])

In [21]:
exog_train= pd.DataFrame(exog_train.astype(float),columns=["exog"])

In [43]:
datetime = pd.to_datetime(wind_df_long["datetime"][:test_begin_point], format = '%Y-%m-%dT%H:%M:%S')

In [34]:
data = pd.concat([datetime,endog_train, exog_train], axis=1)
data.set_index(["datetime"], inplace = True)
data.head()

,endog,exog
datetime,,
2020-01-01 00:00:00,1244.0,204.0
2020-01-01 00:00:00,1188.0,204.0
2020-01-01 01:00:00,1156.0,492.0
2020-01-01 01:00:00,1125.0,492.0
2020-01-01 02:00:00,1106.0,480.0


In [24]:
# import pmdarima as pm
# sarimax = pm.auto_arima(elec_df["embedded_wind_generation"], exogenous=wind_df_long["windspeed"],
#                            start_p=0, start_q=0,
#                            test='adf',
#                            max_p=2, max_q=2, m=12,
#                            start_P=0, seasonal=True,
#                            d=None, D=1, trace=True,
#                            suppress_warnings=True, 
#                            stepwise=True)

In [25]:
data.exog.dtype

dtype('float64')

In [26]:
print(data.dtypes)

datetime    datetime64[ns]
endog              float64
exog               float64
dtype: object


In [27]:
print(data.exog.apply(lambda x: str(x).isnumeric()).sum())

0


In [40]:
model = VAR(data)
# Determine the order of the VAR model using the Akaike information criterion (AIC)


/Users/DocWeaver/.pyenv/versions/3.8.13/envs/lewagon/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [41]:
for i in [1,2,3,4,5,6,7,8,9]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

Lag Order = 1
AIC :  21.69357085001255
BIC :  21.71512743782564
FPE :  2638749219.264986
HQIC:  21.701608488608034 

Lag Order = 2
AIC :  21.59314274034287
BIC :  21.629090170189865
FPE :  2386617175.4476566
HQIC:  21.606546636005117 

Lag Order = 3
AIC :  21.56499630086488
BIC :  21.615350432787466
FPE :  2320379169.1608458
HQIC:  21.58377273219311 

Lag Order = 4
AIC :  21.541000323309472
BIC :  21.6057770459273
FPE :  2265362503.3115773
HQIC:  21.56515558053302 

Lag Order = 5
AIC :  21.510990863297877
BIC :  21.590206093879512
FPE :  2198390683.2147985
HQIC:  21.540531248305538 

Lag Order = 6
AIC :  21.492382428706875
BIC :  21.586052113240925
FPE :  2157861135.085249
HQIC:  21.52731425507634 

Lag Order = 7
AIC :  21.482264401139563
BIC :  21.59040451440607
FPE :  2136138984.69779
HQIC:  21.522593994166982 

Lag Order = 8
AIC :  21.48142114756622
BIC :  21.604047693208255
FPE :  2134339829.8458881
HQIC:  21.527154844295808 

Lag Order = 9
AIC :  21.482271996356097
BIC :  21.61940

In [48]:
mod = model.fit(5)

In [49]:
print(mod.summary())

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Tue, 21, Mar, 2023
Time:                     23:11:15
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                    21.5902
Nobs:                     1470.00    HQIC:                   21.5405
Log likelihood:          -19960.3    FPE:                2.19839e+09
AIC:                      21.5110    Det(Omega_mle):     2.16586e+09
--------------------------------------------------------------------
Results for equation endog
              coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------
const           45.484053        10.160158            4.477           0.000
L1.endog         0.669062         0.025795           25.938           0.000
L1.exog          0.012172         0.012382            0.983           0.326
L2.e

In [50]:
# Forecast
forecast_length = 100
results = mod.get_forecast(forecast_length, exog = exog_test[:forecast_length], alpha=0.05)
forecast = results.predicted_mean
confidence_int = results.conf_int()

AttributeError: 'VARResults' object has no attribute 'get_forecast'

In [ ]:
forecast

In [ ]:
def plot_forecast(fc, train, test, windspeed, upper=None, lower=None):
    is_confidence_int = isinstance(upper, np.ndarray) and isinstance(lower, np.ndarray)
    # Prepare plot series
    fc_series = pd.Series(fc, index=test.index)
    wind_series = pd.Series(windspeed)
    wind_series = wind_series
    lower_series = pd.Series(upper, index=test.index) if is_confidence_int else None
    upper_series = pd.Series(lower, index=test.index) if is_confidence_int else None

    # Plot
    plt.figure(figsize=(10,4), dpi=100)
    plt.plot(train, label='training generation', color='black')
    plt.plot(test, label='actual generation', color='black', ls='--')
    plt.plot(fc_series, label='forecast generation', color='orange')
    plt.plot(wind_series, label='wind speed in hull', color='blue')
    plt.xlim(test_begin_point-200,test_begin_point+200)
    if is_confidence_int:
        plt.fill_between(lower_series.index, lower_series, upper_series, color='k', alpha=.15)
    plt.title('Forecast vs Actuals')
    plt.legend(loc='upper left', fontsize=8);

In [ ]:
endog_test = pd.DataFrame(endog_test)
endog_test.index = endog_test.index + test_begin_point

In [ ]:
forecast_recons = pd.Series(forecast, index=endog_test.index[:forecast_length])

plot_forecast(forecast_recons,endog_train, endog_test, wind_df_long["windspeed"])